In [1]:
# Imports

import polars as pl

In [16]:
cg_ann = pl.read_parquet("../1_snakemake/inputs/annotations/compound_gene.parquet")
print(cg_ann.shape)
meta = pl.read_parquet("../1_snakemake/inputs/metadata/metadata.parquet")
oasis = pl.read_csv("/Users/jewald/repos/2024_03_18_OASIS/2024_03_18_OASIS-data/data/2_reannotated_compiled_list/v5_oasis_03Sept2024_simple.csv")

oasis = oasis.with_columns(
    pl.col("INCHIKEY").str.slice(0,14).alias("INCHI_14")
).select(["OASIS_ID", "INCHI_14"])
cg_ann = cg_ann.with_columns(
    pl.col("inchikey").str.slice(0,14).alias("INCHI_14")
)

cg_ann = cg_ann.join(oasis, on="INCHI_14", how="inner")
print(cg_ann.shape)

(809250, 6)
(283606, 8)


In [17]:
cg_summary = cg_ann.select(["rel_type", "source_id"]).unique()

In [17]:
# Create annotation files from Axiom data
targets = meta.select(["Metadata_Compound", "Metadata_compound_target"]).filter(pl.col("Metadata_compound_target").is_not_null())
pathways = meta.select(["Metadata_Compound", "Metadata_compound_pathway"]).filter(pl.col("Metadata_compound_pathway").is_not_null())


In [18]:
targets = targets.with_columns(
    pl.col("Metadata_compound_target").str.split("; ").alias("Metadata_compound_target")
).explode("Metadata_compound_target").unique()

pathways = pathways.with_columns(
    pl.col("Metadata_compound_pathway").str.split("; ").alias("Metadata_compound_pathway")
).explode("Metadata_compound_pathway").unique()

In [19]:
target_counts = targets.group_by("Metadata_compound_target").count()
pathway_counts = pathways.group_by("Metadata_compound_pathway").count()

Get annotations from ToxCast.
Download latest version of invitrodb from here: https://clowder.edap-cluster.com/datasets/647f712de4b08a6b394e4281?space=647f710ee4b08a6b394e426b

Install mysql on Mac:
brew install mysql@8.0
brew link mysql@8.0 --force
brew services start mysql@8.0
brew services list

Create empty database to extract SQL dump to:
mysql -u root
CREATE DATABASE invitrodb_v4_1;
EXIT;

Unzip the compressed sql dump (expanded size is ~100GB):
gunzip invitrodb_v4_1_mysql.gz

Import into database we created earlier:
mysql -u root invitrodb_v4_1 < invitrodb_v4_1_mysql.sql

Log back in and verify data is there:
mysql -u root
USE prod_internal_invitrodb_v4_1;
SHOW TABLES;


